In [10]:
import os,sys
import pandas as pd
pd.set_option('display.max_rows',10) 
pd.set_option('display.max_columns',20) 
pd.set_option('display.width', None) 
pd.set_option('display.max_colwidth',None)
import matplotlib.pylab as plt
import glob
import numpy as np
import matplotlib as mpl
import yaml
mpl.style.use('default')
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
mpl.rcParams['font.family'] = 'sans-serif'
mpl.rcParams['font.sans-serif'] = 'Arial'
mpl.rcParams['figure.dpi'] = 80
mpl.rcParams['savefig.dpi']=300

Path
```shell
# meQTL (20240713): Permutation
Trans: /anvil/scratch/x-wangwl/02.mQTL/07.tensorqtl/*.SNP_DMRfileter.tsv
Cis: /anvil/scratch/x-wangwl/02.mQTL/04.bulkQTL/04.cis/02.permute/*.filtered.tsv
```

## 1. Download GWAS statistic summary from GWAS Catalog and Processing
https://ftp.ebi.ac.uk/pub/databases/gwas/summary_statistics/harmonised_list.txt

In [2]:
# df=pd.read_csv("https://ftp.ebi.ac.uk/pub/databases/gwas/summary_statistics/harmonised_list.txt",sep='\t',header=None,names=['url'])
# df['real_url']="https://ftp.ebi.ac.uk/pub/databases/gwas/summary_statistics/"+df.url.apply(lambda x:x.lstrip('./')).map(str)
# df.real_url.to_csv("gwas_catalog_harmonised_urls.tsv",header=False,sep='\t',index=False)
# df.head()

```shell
# downloading
rm -f cmd
while read url; do
    # echo ${url}
    echo "wget -nc -P gwas ${url}" >> cmd
    echo "wget -nc -P gwas ${url}-meta.yaml" >> cmd
done < gwas_catalog_harmonised_urls.tsv

cat cmd | parallel -j 24
```

color input format:
- category:

(1): snp,position,MAF,pvalues,N,type='cc', and s(for a case control dataset, the proportion of samples in dataset 1 that are cases) </br1>

test_data1=list(snp='snp1',position=100,MAF=0.998,pvalues=4e-8,N=8742,type='cc',s=26948/(26948+8724)) #beta=2.16 </br1>
print(check_dataset(test_data1)) 

(2): snp,position,beta,varbeta </br1>

test_data1=list(snp='snp1',position=100,beta=2.16,varbeta=0.1,type='cc') #MAF=0.998,N=8742,pvalues=4e-8,
Preferably, give beta and varbeta
print(check_dataset(test_data1))


dataset1=list(beta=effect_size, varbeta=standard_error^2,type="cc",MAF=frequency,N=sample_size,s=cases/sample_size

- Quant

(1): test_data1=list(snp='snp1',position=100,beta=2.16,varbeta=0.1,type='quant',MAF=0.998,N=8742)

(2): test_data1=list(snp='snp1',position=100,type='quant',MAF=0.998,N=8742,pvalues=4e-8)

N: Number of samples in dataset

In [3]:
# load meta
import requests
indir="gwas"
if not os.path.exists("gwas_catalog_harmonised_metadata.tsv"):
    R=[]
    for file in glob.glob(f"{indir}/*-meta.yaml"):
        sname=os.path.basename(file.rstrip('-meta.yaml'))
        # print(file)
        try:
            with open(file) as f:
                # params = yaml.load(f, Loader=yaml.FullLoader)
                params = yaml.safe_load(f)
        except:
            continue
        try:
            sample_info=params['samples'][0]
        except:
            sample_info={}
        url=params.get('gwas_catalog_api',None)
        if not url is None:
            try:
                r=requests.get(url)
                initialSampleSize=r.json()['initialSampleSize']
                case,control=initialSampleSize.split(', ')
                if 'cases' in case.lower() and 'controls' in control.lower():
                    n_case=int(case.split(' ')[0].replace(',',''))
                    n_control=int(control.split(' ')[0].replace(',',''))
            except:
                n_case=None;n_control=None
        else:
            n_case=None;n_control=None
        R.append([sname,params['gwas_id'],params.get('trait_description',''),params.get('ontology_mapping',''),params.get('genome_assembly',''),
                 sample_info.get('sample_size',np.nan),params.get('is_harmonised',np.nan),sample_info.get('case_control_study',np.nan),
                 n_case,n_control,sample_info.get('case_count',np.nan),sample_info.get('control_count',np.nan)])
    meta=pd.DataFrame(R,columns=['filename','gwas_id','trait_description','ontology_mapping','genome_assembly','sample_size','is_harmonised',
                                 'case_control_study','n_case','n_control','case_count','control_count'])
    meta.to_csv("gwas_catalog_harmonised_metadata.tsv",sep='\t',index=False)
else:
    meta=pd.read_csv("gwas_catalog_harmonised_metadata.tsv",sep='\t')
meta

,filename,gwas_id,trait_description,ontology_mapping,genome_assembly,sample_size,is_harmonised,case_control_study,n_case,n_control,case_count,control_count
0,34662886-GCST90080617-EFO_0000284.h.tsv.gz,GCST90080617,['ICD10 N40: Benign prostatic hyperplasia'],['EFO_0000284'],GRCh38,172320.0,True,NaN,15936.0,156384.0,NaN,NaN
1,34662886-GCST90080307-EFO_0000408.h.tsv.gz,GCST90080307,['ICD10 K83.8: Other specified diseases of biliary tract'],['EFO_0009534'],GRCh38,387930.0,True,NaN,1262.0,386668.0,NaN,NaN
2,GCST90265493.h.tsv.gz,GCST90265493,['Urine methionine sulfone levels in chronic kidney disease'],['EFO_0800072'],GRCh38,4888.0,True,False,NaN,NaN,NaN,NaN
3,GCST90266242.h.tsv.gz,GCST90266242,['Plasma X-12680 levels in chronic kidney disease'],['EFO_0021320'],GRCh38,4934.0,True,False,NaN,NaN,NaN,NaN
4,35078996-GCST90089261-EFO_0007937.h.tsv.gz,GCST90089261,['Serum levels of protein RLN3'],['EFO_0007937'],GRCh38,5349.0,True,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
29396,34503513-GCST90060952-EFO_0004530.h.tsv.gz,GCST90060952,['Triacylglycerol(53:3)_[M+NH4]1+ levels'],['EFO_0004530'],GRCh38,5662.0,True,NaN,NaN,NaN,NaN,NaN
29397,32929287-GCST90001566-EFO_0007937.h.tsv.gz,GCST90001566,['Naive CD4-CD8- T cell Absolute Count'],['EFO_0004587'],GRCh38,3395.0,True,NaN,NaN,NaN,NaN,NaN
29398,33303764-GCST90010113-EFO_0007937.h.tsv.gz,GCST90010113,['CD40 ligand'],['EFO_0007937'],GRCh38,1301.0,True,NaN,NaN,NaN,NaN,NaN
29399,33414548-GCST90010726-EFO_0009777.h.tsv.gz,GCST90010726,['Citrulline levels'],['EFO_0009777'],GRCh38,30940.0,True,NaN,NaN,NaN,NaN,NaN


In [4]:
meta.case_control_study.value_counts(dropna=False)

NaN      26489
False     2657
True       255
Name: case_control_study, dtype: int64

In [ ]:
meta=pd.read_csv("gwas_catalog_harmonised_metadata.tsv",sep='\t')
hm_usecols=['hm_rsid','hm_variant_id','hm_chrom','hm_pos','hm_other_allele','hm_effect_allele','hm_effect_allele_frequency','hm_odds_ratio','hm_beta',]
usecols=['rsid','variant_id','chromosome','base_pair_location','other_allele','effect_allele','effect_allele_frequency','odds_ratio','beta']
cols=['rsid','variant_id','chrom','position','other_allele','effect_allele','MAF','odds_ratio','beta']
common_cols=['p_value','standard_error']
outdir="/anvil/projects/x-mcb130189/Wubin/gwas"
for category in ['cc','quant']:
    if category=='cc':
        # category: snp='snp1',position=100,MAF=0.998,pvalues=4e-8,N=8742,type='cc',s=26948/(26948+8724)
        # or snp,position,beta,varbeta
        meta1=meta.loc[(meta.n_case.notna()) & (meta.n_control.notna())]
    else:
        # Quant:snp='snp1',position=100,beta=2.16,varbeta=0.1,type='quant',MAF=0.998,N=8742
        # or: snp='snp1',position=100,type='quant',MAF=0.998,N=8742,pvalues=4e-8
        meta1=meta.loc[(meta.n_case.isna()) & (meta.n_control.isna())]
    for gwas_id,file in meta1.loc[:,['gwas_id','filename']].values:
        if not os.path.exists(os.path.join('gwas',file)):
            continue
        if os.path.exists(os.path.join(outdir,f"gwas_catalog_{category}_no_beta",gwas_id)) or os.path.exists(
            os.path.join(outdir,f"gwas_catalog_{category}_with_beta",gwas_id)):
            os.remove(os.path.join('gwas',file))
            continue
        try:
            df=pd.read_csv(os.path.join('gwas',file),sep='\t')
            n1=len([col for col in df.columns if col in hm_usecols])
            n2=len([col for col in df.columns if col in usecols])
            if n2 > n1:
                df=df.reindex(columns=usecols+common_cols)
            else:
                df=df.reindex(columns=hm_usecols+common_cols)
            df.columns=cols+common_cols
            df['FDR']=df.p_value.astype(float) * df.shape[0] # bonferroni
            df=df.loc[df.FDR <= 0.05]
            if df.shape[0]==0:
                os.remove(os.path.join('gwas',file))
                continue
            # df.insert(0,'gwas_id',gwas_id)
            beta_na_sum=df.beta.notna().sum()
            if beta_na_sum == 0:
                outdir1=os.path.join(outdir,f"gwas_catalog_{category}_no_beta")
                df.drop('beta',axis=1,inplace=True)
                df=df.loc[df.MAF.notna()]
                df=df.loc[df.p_value.notna()]
            else:
                outdir1=os.path.join(outdir,f"gwas_catalog_{category}_with_beta")
            if not os.path.exists(outdir1):
                os.mkdir(outdir1)
            df=df.loc[df.rsid.notna()]
            if df.shape[0]==0:
                os.remove(os.path.join('gwas',file))
                continue
            df.to_csv(os.path.join(outdir1,gwas_id),sep='\t',index=False)
            os.remove(os.path.join('gwas',file))
        except:
            print(file)

/tmp/ipykernel_1399003/3299161378.py:9: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(os.path.join('gwas',file),sep='\t')


In [3]:
# merge results
def parse_chrom(x):
    try:
        return "chr"+str(int(x))
    except:
        return "chr"+str(x)
for category in ['cc','quant']:
    for folder_name in [f"gwas_catalog_{category}_no_beta",f"gwas_catalog_{category}_with_beta"]:
        print(folder_name)
        R=[]
        print(len(os.listdir(os.path.join("gwas",folder_name))))
        for file in os.listdir(os.path.join("gwas",folder_name)):
            df=pd.read_csv(os.path.join("gwas",folder_name,file),sep='\t')
            # print(df.shape[0])
            df.insert(0,'gwas_id',file)
            df.chrom=df.chrom.apply(parse_chrom)
            df.position=df.position.map(int)
            df.rename(columns={'rsid':'snp','p_value':'pvalues'},inplace=True)
            if folder_name.endswith('with_beta'):
                df['varbeta']=df.standard_error.apply(np.sqrt)
            R.append(df)
        df=pd.concat(R)
        df.to_csv(os.path.join("gwas_catalog",f"{folder_name}.tsv.gz"),sep='\t',index=False)

gwas_catalog_cc_no_beta
794
gwas_catalog_cc_with_beta
817
gwas_catalog_quant_no_beta
17
gwas_catalog_quant_with_beta
9358


/tmp/ipykernel_2393043/815725759.py:13: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(os.path.join("gwas",folder_name,file),sep='\t')
/tmp/ipykernel_2393043/815725759.py:13: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(os.path.join("gwas",folder_name,file),sep='\t')
/tmp/ipykernel_2393043/815725759.py:13: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(os.path.join("gwas",folder_name,file),sep='\t')
/tmp/ipykernel_2393043/815725759.py:13: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(os.path.join("gwas",folder_name,file),sep='\t')
/tmp/ipykernel_2393043/815725759.py:13: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(os.path.join("gwas",fol

## 2. Read MAF & snp positions from vcf

In [78]:
! ls /anvil/scratch/x-wangwl/02.mQTL/04.bulkQTL/02.Genotypes/all_auto.vcf.gz

/anvil/scratch/x-wangwl/02.mQTL/04.bulkQTL/02.Genotypes/all_auto.vcf.gz


In [79]:
df=pd.read_csv("/anvil/scratch/x-wangwl/02.mQTL/04.bulkQTL/02.Genotypes/all_auto.vcf.gz",sep='\t',comment="#",usecols=[0,1,2,7])
df.columns=['chrom','pos','rsid','info']
df['varid']=df.chrom.map(str)+'-'+df.pos.map(str)
df['MAF']=df['info'].apply(lambda x:float(x.split('MAF=')[1].split(';')[0]))
df=df.loc[:,['chrom','pos','varid','rsid','MAF']].drop_duplicates()
df.sort_values(['chrom','pos'],ascending=True,inplace=True)
df.to_csv("MAF.tsv",sep='\t',index=False)
# sort 
df.head()

,ID,Chrom,POS,MAF
0,rs147880041,chr10,47317,0.3840
1,rs9329305,chr10,47876,0.4995
2,rs4431953,chr10,48005,0.1670
3,rs145359804,chr10,48219,0.4995
4,rs10904045,chr10,48486,0.0000


## 3. Preprocessing for cis-meQTLs & trans-meQTl (permutation)
Cis: /anvil/scratch/x-wangwl/02.mQTL/04.bulkQTL/04.cis/02.permute/*.filtered.tsv

Trans: /anvil/scratch/x-wangwl/02.mQTL/07.tensorqtl/*.SNP_DMRfileter.tsv

- Quant

(1): test_data1=list(snp='snp1',position=100,beta=2.16,varbeta=0.1,type='quant',MAF=0.998,N=8742)

(2): test_data1=list(snp='snp1',position=100,type='quant',MAF=0.998,N=8742,pvalues=4e-8)

N: Number of samples in dataset

In [2]:
! ls /anvil/scratch/x-wangwl/02.mQTL/04.bulkQTL/04.cis/02.permute/*.filtered.tsv -sh
# N = 110

2.5M /anvil/scratch/x-wangwl/02.mQTL/04.bulkQTL/04.cis/02.permute/B-Mem.permutation.filtered.tsv
2.5M /anvil/scratch/x-wangwl/02.mQTL/04.bulkQTL/04.cis/02.permute/B-Naive.permutation.filtered.tsv
2.0M /anvil/scratch/x-wangwl/02.mQTL/04.bulkQTL/04.cis/02.permute/NK-cell1.permutation.filtered.tsv
2.5M /anvil/scratch/x-wangwl/02.mQTL/04.bulkQTL/04.cis/02.permute/NK-cell2.permutation.filtered.tsv
9.0M /anvil/scratch/x-wangwl/02.mQTL/04.bulkQTL/04.cis/02.permute/Tc-Mem.permutation.filtered.tsv
4.0M /anvil/scratch/x-wangwl/02.mQTL/04.bulkQTL/04.cis/02.permute/Tc-Naive.permutation.filtered.tsv
3.5M /anvil/scratch/x-wangwl/02.mQTL/04.bulkQTL/04.cis/02.permute/Th-Mem.permutation.filtered.tsv
4.0M /anvil/scratch/x-wangwl/02.mQTL/04.bulkQTL/04.cis/02.permute/Th-Naive.permutation.filtered.tsv


In [8]:
! head /anvil/scratch/x-wangwl/02.mQTL/04.bulkQTL/04.cis/02.permute/B-Mem.permutation.filtered.tsv

phe_id	phe_chr	phe_from	phe_to	phe_strd	n_var_in_cis	dist_phe_var	var_id	var_chr	var_from	var_to	dof1	dof2	bml1	bml2	nom_pval	r_squared	slope	adj_emp_pval	adj_beta_pval
dmr492247	chr4	171137902	171137901	+	15931	-765908.0	rs10579886	chr4	170371992.0	170371994.0	105.0	119.581	0.965135	3577.44	3.01516e-05	0.153439	0.407429	0.0329967	0.0340715
dmr492339	chr4	171850035	171850034	+	15154	99940.0	rs333003	chr4	171949974.0	171949974.0	105.0	119.396	0.927341	5477.14	6.64507e-06	0.176484	0.361367	0.0125987	0.012289
dmr492484	chr4	172714094	172714093	+	13965	-198227.0	rs28726202	chr4	172515867.0	172515867.0	105.0	118.597	0.942498	6391.17	9.23566e-06	0.171509	-0.353729	0.0214979	0.0200405
dmr492511	chr4	172802092	172802091	+	14085	-413115.0	rs17058366	chr4	172388977.0	172388977.0	105.0	95.5972	0.898196	3303.98	3.05512e-06	0.188131	0.705038	0.0484952	0.0414824
dmr492561	chr4	173019778	173019777	+	13910	662344.0	rs73004456	chr4	173682121.0	173682121.0	105.0	113.439	0.901351	7196.52	5.20351e-06	0.18

In [3]:
outdir=os.path.abspath("cis-meQTL")
if not os.path.exists(outdir):
    os.mkdir(outdir)

In [4]:
df=pd.read_csv("MAF.tsv",sep='\t',index_col='rsid',dtype={'pos':int})
maf_dict=df.MAF.to_dict()
snp_positions=df.pos.to_dict()
snp_chrom=df.chrom.to_dict()

In [83]:
for file in glob.glob("/anvil/scratch/x-wangwl/02.mQTL/04.bulkQTL/04.cis/02.permute/*.filtered.tsv"):
    group=os.path.basename(file).split('.')[0]
    print(group)
    df=pd.read_csv(file,sep='\t',usecols=['phe_id','var_id','nom_pval']) #'slope','slope_se'
    df.rename(columns={'phe_id':'dmr_id','var_id':'snp','nom_pval':'pvalues'},inplace=True)
    df['MAF']=df.snp.map(maf_dict)
    df['chrom']=df.snp.map(snp_chrom)
    df['position']=df.snp.map(snp_positions)
    df.to_csv(os.path.join(outdir,f"{group}.tsv"),sep='\t',index=False)

B-Mem
Tc-Naive
B-Naive
Th-Naive
Th-Mem
Tc-Mem
NK-cell1
NK-cell2


In [5]:
! ls /anvil/scratch/x-wangwl/02.mQTL/07.tensorqtl/*.SNP_DMRfileter.tsv

/anvil/scratch/x-wangwl/02.mQTL/07.tensorqtl/B-Mem.transQTL.FDRfilter.SNP_DMRfileter.tsv
/anvil/scratch/x-wangwl/02.mQTL/07.tensorqtl/B-Naive.transQTL.FDRfilter.SNP_DMRfileter.tsv
/anvil/scratch/x-wangwl/02.mQTL/07.tensorqtl/Monocyte.transQTL.FDRfilter.SNP_DMRfileter.tsv
/anvil/scratch/x-wangwl/02.mQTL/07.tensorqtl/NK-cell1.transQTL.FDRfilter.SNP_DMRfileter.tsv
/anvil/scratch/x-wangwl/02.mQTL/07.tensorqtl/NK-cell2.transQTL.FDRfilter.SNP_DMRfileter.tsv
/anvil/scratch/x-wangwl/02.mQTL/07.tensorqtl/Tc-Mem.transQTL.FDRfilter.SNP_DMRfileter.tsv
/anvil/scratch/x-wangwl/02.mQTL/07.tensorqtl/Tc-Naive.transQTL.FDRfilter.SNP_DMRfileter.tsv
/anvil/scratch/x-wangwl/02.mQTL/07.tensorqtl/Th-Mem.transQTL.FDRfilter.SNP_DMRfileter.tsv
/anvil/scratch/x-wangwl/02.mQTL/07.tensorqtl/Th-Naive.transQTL.FDRfilter.SNP_DMRfileter.tsv


In [6]:
outdir=os.path.abspath("trans-meQTL")
if not os.path.exists(outdir):
    os.mkdir(outdir)

In [7]:
for file in glob.glob("/anvil/scratch/x-wangwl/02.mQTL/07.tensorqtl/*.SNP_DMRfileter.tsv"):
    group=os.path.basename(file).split('.')[0]
    print(group)
    df=pd.read_csv(file,sep='\t',usecols=['phenotype_id','variant_id','pval']) #'slope','slope_se'
    df.rename(columns={'phenotype_id':'dmr_id','variant_id':'snp','pval':'pvalues'},inplace=True)
    df['MAF']=df.snp.map(maf_dict)
    df['chrom']=df.snp.map(snp_chrom)
    df['position']=df.snp.map(snp_positions)
    df.to_csv(os.path.join(outdir,f"{group}.tsv"),sep='\t',index=False)

B-Naive
Monocyte
NK-cell2
Tc-Naive
NK-cell1
Tc-Mem
Th-Mem
Th-Naive
B-Mem


## 4. Preprocessing for cis-meQTLs & trans-meQTl (norminal)
/anvil/scratch/x-wangwl/02.mQTL/04.bulkQTL/04.cis/00.FDR/*.csv

In [3]:
! ls -sh /anvil/scratch/x-wangwl/02.mQTL/04.bulkQTL/04.cis/00.FDR/*.csv

127M /anvil/scratch/x-wangwl/02.mQTL/04.bulkQTL/04.cis/00.FDR/B-Mem.fdr.csv
112M /anvil/scratch/x-wangwl/02.mQTL/04.bulkQTL/04.cis/00.FDR/B-Naive.fdr.csv
4.3G /anvil/scratch/x-wangwl/02.mQTL/04.bulkQTL/04.cis/00.FDR/Monocyte.fdr.csv
2.5M /anvil/scratch/x-wangwl/02.mQTL/04.bulkQTL/04.cis/00.FDR/NK-cell1.fdr.csv
1.3G /anvil/scratch/x-wangwl/02.mQTL/04.bulkQTL/04.cis/00.FDR/NK-cell2.fdr.csv
 47G /anvil/scratch/x-wangwl/02.mQTL/04.bulkQTL/04.cis/00.FDR/Tc-Mem.fdr.csv
 25G /anvil/scratch/x-wangwl/02.mQTL/04.bulkQTL/04.cis/00.FDR/Tc-Naive.fdr.csv
 22G /anvil/scratch/x-wangwl/02.mQTL/04.bulkQTL/04.cis/00.FDR/Th-Mem.fdr.csv
 30G /anvil/scratch/x-wangwl/02.mQTL/04.bulkQTL/04.cis/00.FDR/Th-Naive.fdr.csv


In [7]:
! head /anvil/scratch/x-wangwl/02.mQTL/04.bulkQTL/04.cis/00.FDR/Th-Naive.fdr.csv

dmrID	pchr	pstart	pend	strand	nvar	dist	vid	vchr	vstart	vend	pval	r2	slope	best	fdr
dmr0	chr1	17453	17452	+	3289	795692	.	chr1	813144	813144	0.00839365	0.0625816	-3.06319	0	0.009094045820125712
dmr0	chr1	17453	17452	+	3289	819316	.	chr1	836768	836768	0.00680565	0.065855	-1.49166	0	0.008079236811731378
dmr0	chr1	17453	17452	+	3289	826633	.	chr1	844085	844085	0.00502703	0.0705884	-59.0295	0	0.006755826919850406
dmr0	chr1	17453	17452	+	3289	830354	.	chr1	847806	847806	0.00917997	0.0611852	-0.599588	0	0.009550839424280826
dmr0	chr1	17453	17452	+	3289	839044	.	chr1	856496	856496	0.00917997	0.0611852	-0.538202	0	0.009550839424280826
dmr0	chr1	17453	17452	+	3289	843344	.	chr1	860796	860796	0.00912118	0.0612854	-0.538662	0	0.009517615571173045
dmr0	chr1	17453	17452	+	3289	848304	.	chr1	865756	865756	9.41439e-06	0.166865	-0.417911	0	5.652347428016576e-05
dmr0	chr1	17453	17452	+	3289	853135	.	chr1	870587	870587	0.00587675	0.0681474	-0.282233	0	0.007416763368714343
dmr0	chr1	17453	17452	+	3289	85

In [4]:
outdir=os.path.abspath("cis-meQTL-norminal")
if not os.path.exists(outdir):
    os.mkdir(outdir)

In [15]:
df=pd.read_csv("MAF.tsv",sep='\t',dtype={'POS':int})
maf_dict=df.set_index('rsid').MAF.to_dict()
vpos2snp=df.set_index(['chrom','pos']).ID.to_dict()

In [ ]:
for file in `ls /anvil/scratch/x-wangwl/02.mQTL/04.bulkQTL/04.cis/00.FDR/*.csv`; do
    bname=$(basename ${file})
    group=${bname/.*}
    echo ${group}
    awk 'BEGIN{FS=OFS="\t"}; FILENAME==ARGV[1]{D1[$3]=$4; D2[$3]=$5; next} FILENAME==ARGV[2]{key=$1"-"$2; print($1,$2,$3,D1[key],D2[key],$4)}' ../MAF.tsv <(awk 'BEGIN {FS=OFS="\t"}; {print($9,$10,$1,$12)}' ${file} | sed '1d') | awk 'BEGIN {FS=OFS="\t"}; {if($4!="") {print($0)}}' > ${group}.tsv
done;

# columns: ['chrom','position','dmr_id','snp','MAF','pvalues']

In [ ]:
# for file in glob.glob("/anvil/scratch/x-wangwl/02.mQTL/04.bulkQTL/04.cis/00.FDR/*.csv"):
#     group=os.path.basename(file).split('.')[0]
#     print(group)
#     outfile=os.path.join(outdir,f"{group}.tsv")
#     if os.path.exists(outfile):
#         os.remove(outfile)
#     for df in pd.read_csv(file,sep='\t',usecols=['dmrID','vchr','vstart','pval'],chunksize=50000): #'slope','slope_se'
#         df.rename(columns={'dmrID':'dmr_id','vchr':'chrom','vstart':'position','pval':'pvalues'},inplace=True)
#         df['snp']=df.apply(lambda x:tuple([x.chrom,x.position]),axis=1).map(vpos2snp)
#         df['MAF']=df.snp.map(maf_dict)
#         df=df.loc[df.snp.notna()]
#         if not os.path.exists(outfile):
#             df.to_csv(outfile,sep='\t',index=False)
#         else:
#             df.to_csv(outfile,sep='\t',index=False,header=False,mode='a')

NK-cell1
Tc-Naive


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f376c465460>>
Traceback (most recent call last):
  File "/anvil/projects/x-mcb130189/Wubin/Software/miniconda3/envs/m3c/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f376c465460>>
Traceback (most recent call last):
  File "/anvil/projects/x-mcb130189/Wubin/Software/miniconda3/envs/m3c/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f376c465460>>
Traceback (most recent call last):
  File "/anvil/projects/x-m

## Preprocessing UCSC GWAS (Skipped)

In [28]:
! ls /gale/netapp/DoD/wenliang/database/ref/hg38/ucsc/

clinvarCnv.bb	 gdcCancer.bb	gwasCatalog.sql
clinvarMain.bb	 gdcCancer.bed	gwasCatalog.trackDb.tab.gz
clinvarMain.bed  genomeinfo.	gwasCatalog.txt.gz


In [21]:
with open("/gale/netapp/DoD/wenliang/database/ref/hg38/ucsc/gwasCatalog.sql",'r') as f:
    lines=f.readlines()
cols=[]
for line in lines[1:-3]:
    cols.append(line.lstrip().split(' ')[0].replace('`',''))
print(len(cols))
print(','.join(cols))
usecols=['chrom','chromStart','chromEnd','name','trait','genes','initSample','replSample','riskAllele','riskAlFreq','pValue','orOrBeta','cnv']

23
bin,chrom,chromStart,chromEnd,name,pubMedID,author,pubDate,journal,title,trait,initSample,replSample,region,genes,riskAllele,riskAlFreq,pValue,pValueDesc,orOrBeta,ci95,platform,cnv


In [22]:
df=pd.read_csv("/gale/netapp/DoD/wenliang/database/ref/hg38/ucsc/gwasCatalog.txt.gz",sep='\t',header=None,names=cols)
df=df.loc[:,usecols]
df.insert(0,'var_id',df.chrom.map(str)+':'+df.chromStart.map(str)+'-'+df.chromEnd.map(str))
df.to_csv("GWAS.tsv",sep='\t',index=False)
df

,var_id,chrom,chromStart,chromEnd,name,trait,genes,initSample,replSample,riskAllele,riskAlFreq,pValue,orOrBeta,cnv
0,chr1:845016-845017,chr1,845016,845017,rs141175086,Morning vs. evening chronotype,LINC01128,"8,724 European ancestry evening chronotype ind...",NaN,rs141175086-C,0.998,4.000000e-08,2.160000,N
1,chr1:946652-946653,chr1,946652,946653,rs2272756,Skin reflectance (Melanin index),NR,720 South Asian ancestry individuals,NaN,rs2272756-A,NR,3.000000e-06,0.324533,N
2,chr1:959138-959139,chr1,959138,959139,rs115438739,California verbal learning test score,NOC2L,"523 European ancestry schizophrenia cases, 100...",NaN,rs115438739-A,NR,7.000000e-06,7.540324,N
3,chr1:959192-959193,chr1,959192,959193,rs13303010,Pancreatic cancer,NOC2L,"9,040 European ancestry cases, 12,496 European...","up to 2,737 cases, up to 4,752 controls",rs13303010-G,0.11,8.000000e-14,1.260000,N
4,chr1:960325-960326,chr1,960325,960326,rs13303327,Heel bone mineral density,KLHL17,"426,824 British ancestry individuals",NaN,rs13303327-G,0.0857658,4.000000e-13,0.023797,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188288,chrY:56944004-56944005,chrY,56944004,56944005,rs306890,Triglycerides,NR,"76,627 European ancestry individuals, 7,795 Hi...",NaN,rs306890-T,0.754,8.000000e-12,0.034000,N
188289,chrY:56944004-56944005,chrY,56944004,56944005,rs306890,Total cholesterol levels,NR,"76,627 European ancestry individuals, 7,795 Hi...",NaN,rs306890-T,NR,1.000000e-22,0.048000,N
188290,chrY:56944004-56944005,chrY,56944004,56944005,rs306890,Total cholesterol levels,NR,"76,627 European ancestry individuals, 7,795 Hi...",NaN,rs306890-T,0.754,7.000000e-19,0.049000,N
188291,chrY:56944004-56944005,chrY,56944004,56944005,rs306890,Low density lipoprotein cholesterol levels,NR,"76,627 European ancestry individuals, 7,795 Hi...",NaN,rs306890-T,NR,4.000000e-19,0.045000,N


In [23]:
print(df.trait.nunique())

4151


## Prepare Input of coloc for cis-mQTL & GWAS